In [1]:
### Data Pre-processing Notebook

## Imports

In [31]:
import sys, os
from facenet_pytorch import MTCNN
from PIL import Image
import torch
import glob
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
import cv2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os
from os.path import join
import argparse
import subprocess
import cv2
from tqdm import tqdm
import argparse
import os
import urllib
import urllib.request
import tempfile
import time
import sys
import json
import random
from tqdm import tqdm
from os.path import join
import matplotlib.pyplot as plt #plotting library
import os #OS module to do os level operation such as mkdir,listsdir,..
from tqdm import tqdm_notebook as tqdm #To show the status bar for iterations
import numpy as np # To handle matrix operations
import cv2 # computer vision library
import dlib # Automatic face tracking library
face_detector = dlib.get_frontal_face_detector() # instantiating face detector class from dlib library 
import joblib # To presist the intermideate files 
import warnings#
warnings.filterwarnings('ignore')# To ignore the unwanted warning messages
from facenet_pytorch import MTCNN
import cv2
from sklearn.model_selection import train_test_split

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import glob
from tqdm.notebook import tqdm
import glob
from tqdm.notebook import tqdm
import pandas as pd
from numpy.random import seed
seed(101)
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import os
import cv2
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras import optimizers
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp
import matplotlib.pylab as plt

%matplotlib inline


In [45]:
DATASET_PATHS = {
    'youtube': 'original_sequences/youtube/c23/',
    'actors': 'original_sequences/actors/c23/',
    'Deepfakes': 'manipulated_sequences/Deepfakes/c23/',
    'Face2Face': 'manipulated_sequences/Face2Face/c23/',
    'FaceShifter': 'manipulated_sequences/FaceShifter/c23/',
    'FaceSwap': 'manipulated_sequences/FaceSwap/c23/',
    'deepfakedetection': 'manipulated_sequences/DeepFakeDetection/'
}


In [ ]:
data_path = "/Volumes/MY PASSPORT/"
output_path = "/Volumes/MY PASSPORT/"
def extract_frames(data_path, output_path):
    reader = cv2.VideoCapture(data_path)
    frame_num = 0
    while reader.isOpened():
        success, image = reader.read()
        if not success:
            print("didn't work")
            break
        cv2.imwrite(join(output_path, '{:04d}.png'.format(frame_num)),
                        image)
        frame_num += 1
        reader.release()
    else:
        raise Exception('Wrong extract frames method: {}'.format(method))


def extract_method_videos(data_path, dataset, compression):
    """Extracts all videos of a specified method and compression in the
    FaceForensics++ file structure"""
    videos_path = join(data_path, DATASET_PATHS[dataset], 'c23', 'videos')
    images_path = join(data_path, DATASET_PATHS[dataset], 'c23', 'images')
    for video in tqdm(os.listdir(videos_path)):
        image_folder = video.split('.')[0]
        extract_frames(join(videos_path, video),
                       join(images_path, image_folder))
for dataset in DATASET_PATHS.keys():
    extract_method_videos(data_path,DATASET_PATHS[dataset],'c23')


## The FastMTCNN Class

The following class implements a strided version of MTCNN. See [here](https://www.kaggle.com/timesler/fast-mtcnn-detector-55-fps-at-full-resolution) for the original implementation.

In [15]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        probs_out = []
        frame_index = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box, prob in zip(boxes[box_ind], probs[box_ind]):
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]].copy())
                probs_out.append(prob)
                frame_index.append(i)
                
        
        return faces, probs, frame_index

## Define face detector

The following face detector can detect all faces in a video in approximately 2.8 seconds, allowing all videos in the public test set to be processed in 2.8 * 4000 = 11200 seconds = 3.1 hours.

In [91]:
fast_mtcnn = FastMTCNN(
    stride=10,
    margin=5,
    factor=0.6,
    keep_all=False,
    device=device,
    thresholds=[0.90, 0.91, 0.98])

In [92]:
from joblib import Parallel, delayed

def f(folder,dataset):
    folder = str(folder)
    inter_path = '/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset] + 'images/' + folder
    for image_ in os.listdir(inter_path):
        img = cv2.imread(os.path.join(inter_path,image_))
        frames = []
        if img is not None:
            frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame))
            filename = image_.split('.')[0]
            if dataset == "youtube" or dataset == "actors":
                output_path = "/Volumes/MY PASSPORT/Base_directory/Real/"
            else:
                output_path = "/Volumes/MY PASSPORT/Base_directory/Fake/"
            save_paths = [output_path+f'{dataset}_{filename}{i}.jpg' for i in range(len(frames))]
            
            mtcnn(frames, save_path=save_paths)

mtcnn = MTCNN(margin=20, keep_all=True, post_process=False)
output_path = '/Volumes/MY PASSPORT/Base_directory/'
"""
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    Parallel(n_jobs=4)(delayed(f)(folder,dataset) for folder in pbar)
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    for folder in pbar:
        Parallel(n_jobs=4)(delayed(f)(folder,dataset))
"""
args = []
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    for folder_ in pbar:
        args.append((folder_,dataset))
Parallel(n_jobs=4)(delayed(f)(i,j) for i,j in args)

100%|██████████| 406/406 [00:00<00:00, 461862.60it/s]


KeyboardInterrupt: 

In [ ]:

for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    for folder_ in pbar:
        args.append((dataset,folder_))
inter_path = '/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset] + 'images/' + folder
Parallel(n_jobs=4)(delayed(f)(i,j) for i,j in args)

In [87]:
args = []
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    for folder_ in pbar:
        args.append((dataset,folder_))
print(args)

KEWB4SM'), ('deepfakedetection', '20_09__podium_speech_happy__RAGUKRMK'), ('deepfakedetection', '20_09__talking_angry_couch__0IRM5ADD'), ('deepfakedetection', '20_09__walking_down_indoor_hall_disgust__0IRM5ADD'), ('deepfakedetection', '20_09__walking_down_indoor_hall_disgust__9OJQX9KV'), ('deepfakedetection', '20_09__walking_down_indoor_hall_disgust__RAGUKRMK'), ('deepfakedetection', '20_09__walk_down_hall_angry__MKEWB4SM'), ('deepfakedetection', '20_09__walk_down_hall_angry__RAGUKRMK'), ('deepfakedetection', '20_11__hugging_happy__GNK9UR8K'), ('deepfakedetection', '20_11__kitchen_still__GNK9UR8K'), ('deepfakedetection', '07_21__hugging_happy__CRGVSDBI'), ('deepfakedetection', '07_21__kitchen_pan__K7KXUHMU'), ('deepfakedetection', '07_21__kitchen_still__K7KXUHMU'), ('deepfakedetection', '07_21__kitchen_still__MKU99DVX'), ('deepfakedetection', '07_21__outside_talking_still_laughing__K7KXUHMU'), ('deepfakedetection', '07_21__secret_conversation__CRGVSDBI'), ('deepfakedetection', '07_21__

In [88]:
for i,j in args:
    print (i)
    print(j)

fakedetection
03_15__exit_phone_room__5C1OMK8W
deepfakedetection
03_15__exit_phone_room__GZNLO4X9
deepfakedetection
03_15__exit_phone_room__OY4I7ZU8
deepfakedetection
03_15__kitchen_pan__5C1OMK8W
deepfakedetection
03_15__kitchen_pan__AIOM1U5V
deepfakedetection
03_15__kitchen_pan__DNUJD8M2
deepfakedetection
04_13__kitchen_pan__00T3UYOR
deepfakedetection
04_13__kitchen_still__00T3UYOR
deepfakedetection
04_13__podium_speech_happy__00T3UYOR
deepfakedetection
04_13__secret_conversation__00T3UYOR
deepfakedetection
04_13__talking_against_wall__00T3UYOR
deepfakedetection
04_13__talking_angry_couch__00T3UYOR
deepfakedetection
04_13__walking_down_street_outside_angry__00T3UYOR
deepfakedetection
04_13__walking_outside_cafe_disgusted__00T3UYOR
deepfakedetection
04_15__kitchen_pan__46TJ9IOJ
deepfakedetection
04_15__kitchen_still__46TJ9IOJ
deepfakedetection
04_15__outside_talking_pan_laughing__46TJ9IOJ
deepfakedetection
15_04__walking_and_outside_surprised__46TJ9IOJ
deepfakedetection
18_04__exit_pho

In [74]:
from joblib import Parallel, delayed

def f(folder,dataset):
    datasets = []
    folder = str(folder)
    inter_path = '/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset] + 'images/'+folder 
    datasets.append(dataset)
    print(dataset)
            
            #mtcnn(frames, save_path=save_paths)

mtcnn = MTCNN(margin=20, keep_all=True, post_process=False)
output_path = '/Volumes/MY PASSPORT/Base_directory/'
datasets = []
file_names = []
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    #datasets.append(dataset)
    #file_names.append(filenames)
    #pbar = tqdm(file_names)
#Parallel(n_jobs=4)(delayed(f)(i,j) for i,j in zip(pbar,datasets))
    pbar = tqdm(filenames)
    for folder in pbar:
        #print(Parallel(n_jobs=4)(delayed(f)(folder,dataset)))
        f(folder,dataset)

ifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifter
FaceShifte

In [50]:
import sys

for i in range(1,10 ):
    sys.stdout.write(str(i))
    sys.stdout.flush()

for i in range(1,10 ):
    print (i)

1234567891
2
3
4
5
6
7
8
9


In [ ]:
Processed_datasets = {
    'Real': '/Volumes/MY PASSPORT/Base_directory/Real/*.jpg',
    'Fake': '/Volumes/MY PASSPORT/Base_directory/Fake/*.jpg'
}

In [ ]:
def read_images(path):
    data = []
    filenames = glob.glob(Processed_datasets[path])
    #pbar = tqdm(filenames)
    #df = pd.DataFrame()
    for image in filenames:
        id = image.split("_")[2]
        id = re.findall(r'\d+',id)[0]
        og_name = image.split("\\")[-1]
        if path == 'Real':
            label = 0
            data.append([id, label,og_name])
        else: 
            label = 1
            data.append([id, label,og_name])
    return data

In [ ]:
data_real = pd.DataFrame(read_images('Real'),columns=["id","label","og_id"])
data_fake = pd.DataFrame(read_images('Fake'),columns=["id","label","og_id"])


In [ ]:
frames = [data_real, data_Deepfakes]
df_data = shuffle(pd.concat(frames))

In [ ]:
class_2 = np.ones(df_data['label'].value_counts()[0])
class_1 = np.zeros(df_data['label'].value_counts()[1])
plt.hist(class_1)
plt.hist(class_2)


In [ ]:
df_data.head()

In [ ]:
df_0 = df_data[df_data['label'] == 0].sample(30000, random_state = 101)
# filter out class 1
df_1 = df_data[df_data['label'] == 1].sample(30000, random_state = 101)

# concat the dataframes
df_data = pd.concat([df_0, df_1], axis=0).reset_index(drop=True)
# shuffle
df_data = shuffle(df_data)

df_data['label'].value_counts()

In [ ]:
y = df_data['label']


In [ ]:
df_train, df_val = train_test_split(df_data, test_size=0.20, random_state=101, stratify=y)


In [ ]:
base_dir = '/Volumes/MY PASSPORT/'
train_dir = os.path.join(base_dir, 'train_dir')

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')


In [ ]:
df_data.set_index('id', inplace=False)


In [ ]:
/Users/asmaaaly/Minerva_Shit/Capstone/Minerva_Capstone/

In [ ]:
from joblib import Parallel, delayed

def f(folder,dataset):
    try:
        folder = str(folder)
        inter_path = 'D:\\'+ DATASET_PATHS[dataset] + 'images\\' + folder
        for image_ in os.listdir(inter_path):
            img = cv2.imread(os.path.join(inter_path,image_))
            frames = []
            if img is not None:
                frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                frames.append(Image.fromarray(frame))
                filename = image_.split('.')[0]
                if dataset == "youtube" or dataset == "actors": 
                    output_path = "'D:\\Base_directory\\Real\\"
                else:
                    output_path = "D:\\Base_directory\\Fake\\"
                save_paths = [output_path+f'{dataset}_{filename}{i}.jpg' for i in range(len(frames))]                        
                mtcnn(frames, save_path=save_paths)
    except Exception as e:
        print(e)

#output_path = '/Volumes/MY PASSPORT/Base_directory/'
mtcnn = MTCNN(margin=20, keep_all=False)
"""
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    Parallel(n_jobs=4)(delayed(f)(folder,dataset) for folder in pbar)
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('/Volumes/MY PASSPORT/'+ DATASET_PATHS[dataset]+ 'images/')
    pbar = tqdm(filenames)
    for folder in pbar:
        Parallel(n_jobs=4)(delayed(f)(folder,dataset))
"""
folders_list = []
dataset_list = []
for dataset in DATASET_PATHS.keys():
    filenames = os.listdir('D:\\'+ DATASET_PATHS[dataset]+ 'images\\')
    for folder_ in filenames:
        if folder_ != '.DS_Store':
            folders_list.append(folder_)
            dataset_list.append(dataset)
pbar_i = tqdm(folders_list)

Parallel(n_jobs=6)(delayed(f)(i,j) for i,j in zip(pbar_i,dataset_list))